# Blight Data Analysis

This analysis is being performed to predict whether or not a person will be in compliance with a blight ticket.

### Load the Libraries

In [493]:
import pandas as pd
import numpy as np
import warnings
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)
    from xgboost import XGBClassifier

### Import the Datasets

In [37]:
data_load = pd.read_csv('https://data.detroitmi.gov/api/views/ti6p-wcg4/rows.csv?accessType=DOWNLOAD&api_foundry=true', engine='python', index_col='Ticket ID')

In [46]:
data_load.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 408330 entries, 445536 to 190940
Data columns (total 39 columns):
Ticket Number                           408330 non-null object
Agency Name                             408330 non-null object
Inspector Name                          408330 non-null object
Violator Name                           408328 non-null object
Violator ID                             408330 non-null int64
Violation Street Number                 408330 non-null int64
Violation Street Name                   408265 non-null object
Violation Zip Code                      76814 non-null object
Mailing Address Street Number           408326 non-null object
Mailing Address Street Name             408324 non-null object
Mailing Address City                    406069 non-null object
Mailing Address State                   405607 non-null object
Mailing Address Zip Code                406067 non-null object
Mailing Address Non-USA Code            2263 non-null object
Mailing

In order to limit the analysis to a smaller dataset, data from the year 2017 will be used.

In [436]:
data_load_2017 = data_load[data_load['Violation Date'].str.contains("2017")]

In [437]:
data_load_2017.shape

(30153, 39)

### Preprocessing the Data

In [438]:
data_load_2017.columns

Index(['Ticket Number', 'Agency Name', 'Inspector Name', 'Violator Name',
       'Violator ID', 'Violation Street Number', 'Violation Street Name',
       'Violation Zip Code', 'Mailing Address Street Number',
       'Mailing Address Street Name', 'Mailing Address City',
       'Mailing Address State', 'Mailing Address Zip Code',
       'Mailing Address Non-USA Code', 'Mailing Address Country',
       'Violation Date', 'Ticket Issued Time', 'Hearing Date', 'Hearing Time',
       'Violation Code', 'Violation Description', 'Disposition', 'Fine Amount',
       'Admin Fee', 'State Fee', 'Late Fee', 'Discount Amount',
       'Clean Up Cost', 'Judgment Amount (Total Due)',
       'Payment Amount (Sum of All Payments)', 'Balance Due',
       'Payment Date (Most Recent)', 'Payment Status', 'Collection Status',
       'Violation Address', 'Violation Parcel ID', 'Violation Latitude',
       'Violation Longitude', 'Violation Location'],
      dtype='object')

In [439]:
pd.options.mode.chained_assignment = None

columns_to_remove = ['Ticket Number', 'Inspector Name', 'Collection Status', 'State Fee', 'Admin Fee',
       'Violator ID', 'Violation Street Number', 'Violation Street Name', 'Mailing Address Street Number',
       'Mailing Address Street Name', 'Mailing Address Zip Code','Mailing Address Country', 'Mailing Address Non-USA Code',
       'Ticket Issued Time', 'Hearing Date', 'Hearing Time','Violation Description', 'Late Fee', 'Discount Amount',
       'Clean Up Cost', 'Judgment Amount (Total Due)', 'Payment Amount (Sum of All Payments)',
       'Payment Date (Most Recent)', 'Violation Address', 'Violation Parcel ID', 'Violation Latitude',
       'Violation Longitude', 'Violation Location','Balance Due','Violation Date']

In [440]:
data_load_2017.drop(columns_to_remove, axis=1, inplace=True)

In [441]:
data_load_2017.head(5)

,Agency Name,Violator Name,Violation Zip Code,Mailing Address City,Mailing Address State,Violation Code,Disposition,Fine Amount,Payment Status
Ticket ID,,,,,,,,,
388072,"Buildings, Safety Engineering & Env Department",Gordon Hawkins,NaN,Detroit,MI,9-1-82(d) - (Dwelling),Responsible by Default,2000.0,NaN
393852,Department of Public Works,BEN IBURA KAILA JOHNSON,48223,DETROIT,MI,22-2-43,Responsible by Default,100.0,NaN
379088,Department of Public Works,DORA HALL,48210,DETROIT,MI,9-1-104,Responsible by Default,50.0,PAID IN FULL
381638,Department of Public Works,MINGZHAO FENG,NaN,DETROIT,MI,22-2-43,Responsible by Default,100.0,PAID IN FULL
401809,Department of Public Works,CEDRIC DAVIS,48219,DETROIT,MI,22-2-83(a)(b)(c),Responsible by Admission,200.0,PAID IN FULL


#### Decide out what to do with the NaN Values

In [442]:
data_load_2017.isnull().sum()

Agency Name                  0
Violator Name                1
Violation Zip Code       13382
Mailing Address City       507
Mailing Address State      507
Violation Code               0
Disposition                269
Fine Amount                  0
Payment Status           15422
dtype: int64

In [443]:
data_load_2017 = data_load_2017[data_load_2017['Payment Status'] != 'NO PAYMENT DUE']

In [444]:
data_load_2017['Agency Name'].value_counts()

Department of Public Works                        11798
Buildings, Safety Engineering & Env Department     9897
Detroit Police Department                           697
Name: Agency Name, dtype: int64

In [445]:
data_load_2017['Payment Status'].fillna(0, inplace=True)
data_load_2017['Disposition'].fillna('None', inplace=True)
data_load_2017['Violation Zip Code'].fillna(data_load_2017['Violation Zip Code'].mode()[0], inplace=True)
data_load_2017.dropna(axis=0, inplace=True)

data_load_2017.isnull().sum()

Agency Name              0
Violator Name            0
Violation Zip Code       0
Mailing Address City     0
Mailing Address State    0
Violation Code           0
Disposition              0
Fine Amount              0
Payment Status           0
dtype: int64

In [446]:
data_load_2017 = data_load_2017.replace({'PARTIAL PAYMENT APPLIED': 0, 'PAID IN FULL':1})

In [447]:
data_load_2017['Mailing Address City'] = data_load_2017['Mailing Address City'].str.upper()
data_load_2017['Violator Name'] = data_load_2017['Violator Name'].str.upper()

In [448]:
data_load_2017.head()

,Agency Name,Violator Name,Violation Zip Code,Mailing Address City,Mailing Address State,Violation Code,Disposition,Fine Amount,Payment Status
Ticket ID,,,,,,,,,
388072,"Buildings, Safety Engineering & Env Department",GORDON HAWKINS,48228,DETROIT,MI,9-1-82(d) - (Dwelling),Responsible by Default,2000.0,0
393852,Department of Public Works,BEN IBURA KAILA JOHNSON,48223,DETROIT,MI,22-2-43,Responsible by Default,100.0,0
379088,Department of Public Works,DORA HALL,48210,DETROIT,MI,9-1-104,Responsible by Default,50.0,1
381638,Department of Public Works,MINGZHAO FENG,48228,DETROIT,MI,22-2-43,Responsible by Default,100.0,1
401809,Department of Public Works,CEDRIC DAVIS,48219,DETROIT,MI,22-2-83(a)(b)(c),Responsible by Admission,200.0,1


In [449]:
data_load_2017['Mailing Address City'] = data_load_2017['Mailing Address City'].str.rstrip()
data_load_2017['Violation Code'] = data_load_2017['Violation Code'].str.rstrip()

#### Creating Features that can be Used in the Analysis

In [450]:
data_load_2017.loc[data_load_2017['Mailing Address City'] == 'DETROIT', 'in_Detroit'] = 1

In [451]:
data_load_2017.loc[data_load_2017['Mailing Address State'] == 'MI', 'in_Michigan'] = 1

In [452]:
data_load_2017.describe()

,Fine Amount,Payment Status,in_Detroit,in_Michigan
count,21939.000000,21939.000000,11283.0,18733.0
mean,337.206436,0.301244,1.0,1.0
std,431.579594,0.458809,0.0,0.0
min,1.000000,0.000000,1.0,1.0
25%,100.000000,0.000000,1.0,1.0
50%,200.000000,0.000000,1.0,1.0
75%,500.000000,1.000000,1.0,1.0
max,10000.000000,1.000000,1.0,1.0


Set bins for "Fine Amount" in order to make it categorical:
* low: 0-150
* medium: 150-250
* high: 250 and up

In [453]:
data_load_2017['Fine Amount Bins'] = pd.cut(data_load_2017['Fine Amount'], [0, 150, 250,50000], labels=['low', 'medium','high'])

In [454]:
data_load_2017.loc[data_load_2017['Violator Name'].str.contains('LLC') | data_load_2017['Violator Name'].str.contains(' INC'), 'is_LLC'] = 1

In [455]:
zipcodes = ['48228','48224','48219','48227']
for zipcode in zipcodes:
    data_load_2017.loc[data_load_2017['Violation Zip Code'] == zipcode, '%s_ZC'%(zipcode)] = 1


dispositions = ['Responsible by Default','Responsible by Admission','Responsible by Determination']
short_disps = ['Default','Admission','Determination']
for disposition,short_disp in zip(dispositions,short_disps):
    data_load_2017.loc[data_load_2017['Disposition'] == disposition, 'Disp_%s'%(short_disp)] = 1

codes = ['22-2-88(b)','9-1-104','9-1-36(a)','9-1-81(a)']
short_codes = ['22-2-88','9-1-104','9-1-36','9-1-81']
for code,short_code in zip(codes,short_codes):
    data_load_2017.loc[data_load_2017['Violation Code'] == code, 'Code_%s'%(short_code)] = 1
    
fine_bins = ['low','medium','high']
for fine_bin in fine_bins:
    data_load_2017.loc[data_load_2017['Fine Amount Bins'] == fine_bin, '%s_Fine_Amount'%(fine_bin)] = 1
    
departments = ['Department of Public Works','Buildings, Safety Engineering & Env Department','Detroit Police Department']
departments_short = ['DPW','BSEED','DPD']
for department,department_short in zip(departments,departments_short):
    data_load_2017.loc[data_load_2017['Agency Name'] == department, '%s_Agency'%(department_short)] = 1


In [456]:
data_load_2017.columns

Index(['Agency Name', 'Violator Name', 'Violation Zip Code',
       'Mailing Address City', 'Mailing Address State', 'Violation Code',
       'Disposition', 'Fine Amount', 'Payment Status', 'in_Detroit',
       'in_Michigan', 'Fine Amount Bins', 'is_LLC', '48228_ZC', '48224_ZC',
       '48219_ZC', '48227_ZC', 'Disp_Default', 'Disp_Admission',
       'Disp_Determination', 'Code_22-2-88', 'Code_9-1-104', 'Code_9-1-36',
       'Code_9-1-81', 'low_Fine_Amount', 'medium_Fine_Amount',
       'high_Fine_Amount', 'DPW_Agency', 'BSEED_Agency', 'DPD_Agency'],
      dtype='object')

In [457]:
data_load_2017.drop(['Fine Amount Bins'], axis=1, inplace=True)

In [458]:
data_load_2017.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21939 entries, 388072 to 382036
Data columns (total 29 columns):
Agency Name              21939 non-null object
Violator Name            21939 non-null object
Violation Zip Code       21939 non-null object
Mailing Address City     21939 non-null object
Mailing Address State    21939 non-null object
Violation Code           21939 non-null object
Disposition              21939 non-null object
Fine Amount              21939 non-null float64
Payment Status           21939 non-null int64
in_Detroit               11283 non-null float64
in_Michigan              18733 non-null float64
is_LLC                   7744 non-null float64
48228_ZC                 10667 non-null float64
48224_ZC                 1307 non-null float64
48219_ZC                 1281 non-null float64
48227_ZC                 1008 non-null float64
Disp_Default             18182 non-null float64
Disp_Admission           2054 non-null float64
Disp_Determination       1435 non-n

In [459]:
data_load_2017.fillna(value=0, axis=0, inplace=True)

In [460]:
for zipcode in zipcodes:
    data_load_2017.loc[data_load_2017['Violation Zip Code'] == zipcode, 'other_ZC'] = 0.0

for disposition,short_disp in zip(dispositions,short_disps):
    data_load_2017.loc[data_load_2017['Disposition'] == disposition, 'Disp_other'] = 0.0

for code,short_code in zip(codes,short_codes):
    data_load_2017.loc[data_load_2017['Violation Code'] == code, 'Code_other'] = 0.0

In [462]:
data_load_2017.fillna(1.0,inplace=True)

In [463]:
feature_creation_drop = ['Agency Name', 'Violator Name', 'Violation Zip Code',
       'Mailing Address City', 'Mailing Address State',
       'Violation Code', 'Disposition', 'Fine Amount']

In [464]:
data_load_2017.drop(feature_creation_drop, axis=1, inplace=True)

In [465]:
data_load_2017.head(20)

,Payment Status,in_Detroit,in_Michigan,is_LLC,48228_ZC,48224_ZC,48219_ZC,48227_ZC,Disp_Default,Disp_Admission,...,Code_9-1-81,low_Fine_Amount,medium_Fine_Amount,high_Fine_Amount,DPW_Agency,BSEED_Agency,DPD_Agency,other_ZC,Disp_other,Code_other
Ticket ID,,,,,,,,,,,,,,,,,,,,,
388072,0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
393852,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
379088,1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
381638,1,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
401809,1,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
392090,0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
405487,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
403023,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
403025,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0


### Analysis of the Data

First want to split the data into train and test data

In [477]:
y = data_load_2017['Payment Status']
X = data_load_2017.drop('Payment Status', axis=1)

In [482]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

First try a Random Forest Classifier:

In [484]:
rf = RandomForestClassifier(n_estimators = 10,
                            random_state = 1,
                            max_depth = 3)

rf.fit(X_train, y_train)
acc_train_rf = rf.score(X_train, y_train)
acc_test_rf = rf.score(X_test, y_test)

print('Training set accuracy: ', acc_train_rf)
print('Test set accuracy: ', acc_test_rf)

Training set accuracy:  0.811805595122785
Test set accuracy:  0.8167730173199635


Try boosting next:

In [494]:
boost_model = XGBClassifier()
boost_model.fit(X_train, y_train)
acc_train_boost = boost_model.score(X_train, y_train)
acc_test_boost = boost_model.score(X_test, y_test)

print('Training set accuracy: ', acc_train_boost)
print('Test set accuracy: ', acc_test_boost)

Training set accuracy:  0.8179590906501054
Test set accuracy:  0.8217866909753874


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
